## from 8.4 out-of-core learning

In [4]:
import numpy as np
import re
from nltk.corpus import stopwords

In [5]:
stop = stopwords.words('english')

In [6]:
def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                           text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) \
        + ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

def stream_docs(path):
    with open(path, 'r', encoding='utf-8') as csv:
        next(csv)
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label
            
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        pass
    return docs, y

In [7]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier

vect = HashingVectorizer(decode_error='ignore',
                         n_features=2**21,
                         preprocessor=None,
                         tokenizer=tokenizer)
clf = SGDClassifier(loss='log', random_state=1, max_iter=1)
doc_stream = stream_docs(path='movie_data.csv')

In [8]:
import pyprind
pbar = pyprind.ProgBar(45, stream=sys.stderr)
classes = np.array([0, 1])
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:29


In [9]:
X_test, y_test = get_minibatch(doc_stream, size=5000)
X_test = vect.transform(X_test)
print('정확도: %.3f' % clf.score(X_test, y_test))

정확도: 0.868


In [10]:
clf = clf.partial_fit(X_test, y_test)

# 9. 웹 애플리케이션에 머신 러닝 모델 내장

In [11]:
import pickle
import os
dest = os.path.join('movieclassifier', 'pkl_objects')
if not os.path.exists(dest):
    os.makedirs(dest)

pickle.dump(stop,
            open(os.path.join(dest, 'stopwords.pkl'), 'wb'),
            protocol=4)
pickle.dump(clf,
            open(os.path.join(dest, 'classifier.pkl'), 'wb'),
            protocol=4)

## 9.2 데이터를 저장하기 위해 SQLite 데이터베이스 설정

* movieclassifier directory 내에서 하기

In [1]:
import sqlite3

In [2]:
conn = sqlite3.connect('reviews.sqlite')
c = conn.cursor()
c.execute('DROP TABLE IF EXISTS review.db')
c.execute('CREATE TABLE review_db'\
    ' (review TEXT, sentiment INTEGER, date TEXT) ')

In [3]:
example1 = 'I love this movie'
c.execute("INSERT INTO review_db"\
    " (review, sentiment, date) VALUES"\
        " (?, ?, DATETIME('now')) ", (example1, 1))

In [4]:
example2 = 'I disliked this movie'
c.execute("INSERT INTO review_db"\
    " (review, sentiment, date) VALUES"\
        " (?, ?, DATETIME('now')) ", (example2, 0))

In [5]:
conn.commit()
conn.close()